In [33]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.decomposition import PCA
import pickle
import matplotlib.pyplot as plt
import gzip
from sklearn.preprocessing import StandardScaler

In [34]:
#dataset reading code is used as taught in the tutorial
def read_image_data(file):
    with gzip.open(file, 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        image_count = int.from_bytes(f.read(4), 'big')
        row_count = int.from_bytes(f.read(4), 'big')
        column_count = int.from_bytes(f.read(4), 'big')
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape((image_count, row_count, column_count))
        return images
def read_label_data(file):
    with gzip.open(file, 'r') as f:
        magic_number = int.from_bytes(f.read(4), 'big')
        label_count = int.from_bytes(f.read(4), 'big')
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels
train_x = read_image_data("./data/mnist/mnist/train-images-idx3-ubyte.gz")
train_y = read_label_data("./data/mnist/mnist/train-labels-idx1-ubyte.gz")
test_x = read_image_data("./data/mnist/mnist/t10k-images-idx3-ubyte.gz")
test_y = read_label_data("./data/mnist/mnist/t10k-labels-idx1-ubyte.gz")

In [35]:
train_x_reshaped = train_x.reshape((train_x.shape[0], train_x.shape[1]*train_x.shape[2]))
test_x_reshaped = test_x.reshape((test_x.shape[0], test_x.shape[1]*test_x.shape[2]))

In [36]:
train_x_reshaped = StandardScaler().fit_transform(train_x_reshaped)
test_x_reshaped = StandardScaler().fit_transform(test_x_reshaped)

In [37]:
def compute_accuracy(y_true, y_pred):
    correct_classif = 0
    total = len(y_true)
    for i in range(total):
        if(y_true[i] == y_pred[i]):
            correct_classif += 1
    return correct_classif / total

In [38]:
best_n_comp = 15
pca = PCA(n_components=best_n_comp)
pca_transformed_train_x = pca.fit_transform(train_x_reshaped)
pca_transformed_test_x = pca.transform(test_x_reshaped)
print(f"Transformed train_x shape : {pca_transformed_train_x.shape}")
print(f"Tranformed test_x shape : {pca_transformed_test_x.shape}")

Transformed train_x shape : (60000, 15)
Tranformed test_x shape : (10000, 15)


In [39]:
def get_class_wise_data(train_x, train_y):
    class_wise_data = []
    
    for label in range(10):
        # print(label)
        label_idx = list(np.where(train_y == label)[0])
        # print(f"label_idx = {len(label_idx)}")
        class_x_data = train_x[label_idx].T
        class_wise_data.append(class_x_data)
    total_X_matrix = np.array(class_wise_data[0])
    for label in range(1, 10):
        total_X_matrix = np.concatenate((total_X_matrix, class_wise_data[label]), axis=1)
        
    return class_wise_data, total_X_matrix

def get_class_wise_means(class_wise_x_train):
    class_wise_means = []
    for i in range(len(class_wise_x_train)):
        mean_x_i = np.mean(class_wise_x_train[i], axis=1)
        mean_x_i = mean_x_i.reshape(mean_x_i.shape[0], 1)
        class_wise_means.append(mean_x_i)
    return class_wise_means

def get_S_w(class_wise_x_train, class_wise_means):
    num_classes = len(class_wise_x_train)
    num_features = class_wise_means[0].shape[0]
    S_mats = []
    S_w = np.zeros((num_features, num_features))
    for i in range(num_classes):
        S_i = np.dot((class_wise_x_train[i] - class_wise_means[i]), (class_wise_x_train[i] - class_wise_means[i]).T)
        S_mats.append(S_i)
        S_w += S_i
    return S_w

def get_S_b(total_x_train, S_w):
    mean_X_total = np.mean(total_x_train, axis=1)
    mean_X_total = mean_X_total.reshape((mean_X_total.shape[0], 1))
    S_t = np.dot((total_x_train - mean_X_total), (total_x_train - mean_X_total).T)
    S_b = S_t - S_w
    return S_b

def sort_eigen_vects(eigenvectors, eigenvalues):
    val_to_vect = []
    for i in range(len(eigenvalues)):
        evec = eigenvectors[i, :]
        val_to_vect.append((eigenvalues[i], evec))
    sorted_vals = sorted(val_to_vect, key=lambda x : x[0], reverse=True)
    sorted_evec = np.column_stack([x[1] for x in sorted_vals])
    return sorted_evec

def perform_FDA(train_x, train_y):
    class_wise_x_train, total_x_train = get_class_wise_data(train_x, train_y)
    class_wise_means = get_class_wise_means(class_wise_x_train)
    S_w = get_S_w(class_wise_x_train, class_wise_means)
    S_b = get_S_b(total_x_train, S_w)
    scat_prod = np.dot(np.linalg.inv(S_w), S_b)
    un, eigenvalues, eigenvectors = np.linalg.svd(scat_prod)
    W = sort_eigen_vects(eigenvectors, eigenvalues)[:, :9]
    return W
def project_data(X, W):
    return np.dot(W.T, X)


In [40]:
W_fda = perform_FDA(pca_transformed_train_x, train_y)

In [41]:
projected_X_train = project_data(pca_transformed_train_x.T, W_fda)
projected_X_test = project_data(pca_transformed_test_x.T, W_fda)

In [42]:
clf = LDA()
clf.fit(projected_X_train.T, train_y)
pickle.dump(clf, open(f'./models/q4_lda.sav', 'wb'))
y_preds = clf.predict(projected_X_test.T)

In [43]:
labels = [x for x in range(10)]
overall_accuracy = compute_accuracy(test_y, y_preds)
print(f"\nOverall accuracy = {overall_accuracy}\n\n-------------\n")
# #Class-wise-accuracy
class_wise_accuracy = {}
for lab in labels:
    class_idxs = np.where(test_y == lab)[0]
    true_labs = test_y[class_idxs]
    pred_labs = y_preds[class_idxs]
    class_wise_accuracy[lab] = compute_accuracy(true_labs, pred_labs)
for lab in class_wise_accuracy.keys():
    print(f"Class-wise accuracy for class-{lab} : {class_wise_accuracy[lab]}")


Overall accuracy = 0.7645

-------------

Class-wise accuracy for class-0 : 0.7642857142857142
Class-wise accuracy for class-1 : 0.960352422907489
Class-wise accuracy for class-2 : 0.7315891472868217
Class-wise accuracy for class-3 : 0.7930693069306931
Class-wise accuracy for class-4 : 0.7627291242362525
Class-wise accuracy for class-5 : 0.624439461883408
Class-wise accuracy for class-6 : 0.8361169102296451
Class-wise accuracy for class-7 : 0.7801556420233463
Class-wise accuracy for class-8 : 0.6735112936344969
Class-wise accuracy for class-9 : 0.6788899900891973
